# Instruction Finetuning using IA3

This notebook looks into how to perform instruction finetuning using IA3 PEFT method. The task is to perform Supervised finetuning (SFT) of Mistral for Natural language to SQL Query generation task.

In [ ]:
import os
os.environ["WANDB_PROJECT"]="mistral_instruct_finetuning"

from enum import Enum
from functools import partial
import pandas as pd
import torch
import json

from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig, set_seed
from datasets import load_dataset
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from peft import get_peft_model, IA3Config, TaskType

seed = 42
set_seed(seed)